<a href="https://colab.research.google.com/github/himanshu-nakrani/TMLC-Gen-ai-projects/blob/main/Langchain_Chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain cohere langchain-huggingface bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 75.0 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
from huggingface_hub import login
login(userdata.get('huggingface_login'))

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer,BitsAndBytesConfig,AutoModelForCausalLM
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_huggingface.llms import HuggingFacePipeline

In [ ]:
model_name='mistralai/Mistral-7B-v0.1'

model_config = transformers.AutoConfig.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

use_4bit = True

bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)

text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.1,
    max_new_tokens=512,
    output_scores=True
)

mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0


## Sequential Chain

In [ ]:
template = """
I want you to act as a expert who can generate details on the given scientific process {topic}.
"""

prompt_template = PromptTemplate(
    input_variables=["topic"],
    template=template,
)

chain = LLMChain(llm=mistral_llm, prompt=prompt_template)

In [ ]:
description = "Photosynthesis"
prompt_template.format(topic=description)

'\nI want you to act as a expert who can generate details on the given scientific process Photosynthesis.\n'

In [ ]:
print(chain.invoke(input={'topic':description}))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:818: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_scores` is. When `return_dict_in_generate` is not `True`, `output_scores` is ignored.
  warnings.warn(


{'topic': 'Photosynthesis', 'text': '\nI want you to act as a expert who can generate details on the given scientific process Photosynthesis.\n\nThe process of photosynthesis is a process that is carried out by plants and other organisms. It is a process that is used to convert light energy into chemical energy. The process of photosynthesis is a process that is used to convert light energy into chemical energy.\n\nThe process of photosynthesis is a process that is used to convert light energy into chemical energy. The process of photosynthesis is a process that is used to convert light energy into chemical energy.\n\nThe process of photosynthesis is a process that is used to convert light energy into chemical energy. The process of photosynthesis is a process that is used to convert light energy into chemical energy.\n\nThe process of photosynthesis is a process that is used to convert light energy into chemical energy. The process of photosynthesis is a process that is used to conver